# Lab Assignment Six: Wide and Deep Networks



__*Austin Chen, Luke Hansen, Oscar Vallner*__

## 1. Preparation and Overview


### 1.1 Business Understanding

For the 6th Lab Assignment, we will be using the "Diabetes 130-US hospitals of years 1999-2008" dataset, downloaded from the UCI Machine Learning Repository. This dataset has been prepared for usage in analyzing medical factors related to hospital readmission for patients with diabetes. The dataset represents ten years worth of clinical care data from 130 hospitals in the United States, as well as integrated delivery networks. In order for an entry to qualify for the dataset, it must satisfy the following requirements:

1. It must be a hospital admission
2. The admission must be diabetic related
3. The length of the stay must be more than 1 day but less than 2 weeks.
4. Laboratory tests must have been performed on the patient
5. Medications must have been administered or prescribed.

The dataset's features comprise of a mix of nominal and numeric datatypes. The features span a multitude of patient information categories such as race, gender, age, weight, and time spent in hospital. Some of the more specific features include information such as medications administered (miglitol, metformin, tolazamide, etc) and number of lab procedures. 

The features and content of this dataset can provide extremely important information for the healthcare industry, for various reasons. First and foremost, a classification model that can accurately predict whether a diabetic inpatient will return to the hospital has several implications in itself. On the medical side, knowing that a patient is at risk for rehospitalization, given a patient's specific attributes, can help doctors and physicians modify their treatment methodology to minimize the chance of rehospitalization. A similar concept applies for the patients themselves; patients who can know they may be at risk for rehospitalization can take extra precautions once they are discharged in an attempt to prevent further emergencies. And finally, if hospital administrators know that a certain number of patients will be readmitted, they could use this information in order to better plan room reservations, medical supply projections, and further logistical planning. Though it is difficult to plan resources for hospitals due to their hectic nature, having the information is still there nonetheless, and provides more options. In any case, the business applications are wide and deep no matter the stakeholder.

With the aforementioned data, we plan to construct a classifier that will predict whether or not a patient will be rehospitalized within 30 days after discharge.

However, we must highlight an important issue with our classification task. The original dataset creates three distinct class boundaries for hospital readmittance:

1. NO: No record of readmission
2. &lt;30: If the patient was readmitted in less than 30 days.
3. &gt;30: If the patient was readmitted in more than 30 days.

For the sake of our business case, we believe it would be far more feasible to narrow the scope to a binary classification. Instead of the three distinct classes seen above, a simplification could prove more pratical and informative:

1. NO: No record of readmission
2. &lt;30: If the patient was readmitted in less than 30 days.

We have decided to eliminate the third type above (>30 days) for a number of reasons. First, it is important to reiterate that one of our primary business cases is to help medical professionals improve their treatment methodologies on diabetic hospital patients. Once a patient has been released for more than 30 days, the more difficult it becomes to blame any rehospitalizations on the aspects of the procedure itself. Pinpointing reasons for rehopsitalization becomes far more nebulous once a significant amount of time has passed since original hospitalization. There could be several extenuating external circumstances that contribute to a rehospitalization of a patient independent of the hospital's treatment. Furthermore, limiting the scope to a binary classification can greatly enhance our ability to choose the most appropriate evaluation method. For an industry as important as healthcare, especially where human life is at risk, we believe that a simpler classification, with better evaluation practices and better performance, is more deployable than a multi-class problem with lower performance. Because the mere concept of emergency hospitalization bears the risk of human mortality, we would want to achieve an accuracy of at least 95+%. **However, as we will discuss later, pure accuracy will not be our primary evaluation metric.**

With more time, we can continue to refine our classification model to accomodate more classes. With regards to our specific dataset, we would require consultation with a domain expert in order to evaluate the extenuating circumstances that contribute to rehospitalization past 30 days. 

---

Link to dataset: http://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008#

---

### 1.2 Class Variables

### Features

Here is a full guide to the features found in our dataset. A more comprehensive feature guide can be found here: https://www.hindawi.com/journals/bmri/2014/781670/tab1/

<br> </br>

| No. Feature     | Feature Name  | Feature Type    | Description     |
| :-------------: |:-------------:| :-------------: | :-------------: |
| ~~1~~ | ~~encounter_id~~ | ~~Numeric~~ | *Removed from dataset for model* |
| ~~2~~ | ~~patient_nbr~~ | ~~Numeric~~ | *Removed from dataset for model* |
| 3 | race | Nominal | Race: Caucasian, Asian, Hispanic, etc. | 
| 4 | gender | Nominal| Gender: male, female, unknown/invalid |
| 5 | age | Nominal | Age ranges in 10-year intervals: [0, 10), [10, 20) |
| ~~6~~ | ~~weight~~ | ~~Numeric~~ | *Removed from dataset for model* |
| 7 | admission_type_id | Nominal | Integer ientifier. Ex. Emergency, elective, etc. |
| 8 | discharge_disposition_id | Nominal | Integer identifier. Ex. Discharged to home, expired, etc. |
| 9 | admission_source_id | Nominal | Integer identifier |
| 10 | time_in_hospital | Numeric | Integer number of days spent in hospital |
| ~~11~~ | ~~payer_code~~ | ~~Nominal~~ | *Removed from dataset for model* |
| ~~12~~ | ~~medical_specialty~~ | ~~Nominal~~ | *Removed from dataset for model* |
| 13 | num_lab_procedures | Numeric | Number of lab tests performed during admission |
| 14 | num_procedures | Numeric | Number of other procedures performed during admission |
| 15 | num_medications | Numeric | Number of distinct medications administered during encounter |
| 16 | num_outpatient | Number | Number of outpatient visits of the patient in the year preceding the encounter |
| 17 | number_emergency | Number | Number of emergency visits of the patient in the year preceding the encounter |
| 18 | number_inpatient | Number | Number of impatient visits of the patient in the year preceding the encounter |
| 19 | diag_1 | Nominal | Primary diagnosis |
| 20 | diag_2 | Nominal | Secondary diagnosis |
| 21 | diag_3 | Nominal| Any additional secondary diagnosis |
| 22 | number_diagnoses | Numeric | Number of diagnoses |
| 23 | max_glu_serum | Nominal |  Result for glucose serum test. Represented by a range of values. |
| 24 | A1Cresult | Nominal | Result for A1c test. Represented by a arange of values. |
| 25-48 | Medications | Nominal | 24 features for different medications |
| 49 | change | Nominal | Whether there was a change in diabetic medications. |
| 50 | diabetesMed | Nominal | Indicates whether any diabetic medication was prescribed. |
| 51 | Readmitted | Nominal | Whether patient was readmitted. "No" or "less than 30 days". |

<br> </br>

Though we removed some features such as encounter_id and patient_nbr because they don't contribute any relevant information to our classification dataset, we also decided to remove a few other features that actually have conceptual significance to the dataset. For example, the weight attribute has many potential ties to qualifying overall health. However, 97% of the weight data was missing, and imputing 97% of the data for a category is not possible.

### 1.3 Data Preparation

We begin our data preparation by reading in our dataset.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
df = pd.read_csv('data/diabetic_data.csv', encoding = 'latin1',low_memory=False)

df.head()

First, the most important aspect of our data preparation is to narrow down our classification problem to binary classification, as we discussed above. We will be dropping all entries in which patients were readmitted over 30 days past their initial discharge. We will be changing all instances of "NO" (no readmittance) to 0 and all instances of "&lt;30" to 1.

In [ ]:
df.readmitted[df.readmitted == 'NO' ] = 0
df.readmitted[df.readmitted == '<30' ] = 1
df = df.drop(df[df.readmitted == '>30'].index)

Next, we will drop all features that are unnecessary for our classification model. Furthermore, we will be dropping entries in which there is unimputable data.

In [ ]:
from sklearn.model_selection import train_test_split

df.drop(['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty'], axis=1, inplace=True)
df.dropna(axis=1, how='all')
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['readmitted'])

Then, we must encode all the categorical features as integers.

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
int_categorical_headers = [
'admission_type_id',
'discharge_disposition_id',
'admission_source_id',
]
numeric_headers = [
    "time_in_hospital", 
    "num_lab_procedures", 
    "num_procedures", 
    "num_medications", 
    "number_outpatient", 
    "number_emergency", 
    "number_inpatient", 
    "number_diagnoses",
]
categorical_headers = [
 'diag_3',
 'diag_2',
 'diag_1',
 'race',
 'gender',
 'age',
 'max_glu_serum',
 'A1Cresult',
 'metformin',
 'repaglinide',
 'nateglinide',
 'chlorpropamide',
 'glimepiride',
 'acetohexamide',
 'glipizide',
 'glyburide',
 'tolbutamide',
 'pioglitazone',
 'rosiglitazone',
 'acarbose',
 'miglitol',
 'troglitazone',
 'tolazamide',
 'examide',
 'citoglipton',
 'insulin',
 'glyburide-metformin',
 'glipizide-metformin',
 'glimepiride-pioglitazone',
 'metformin-rosiglitazone',
 'metformin-pioglitazone',
 'change',
 'diabetesMed']

df_train.replace(to_replace=' ?',value=np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.reset_index()

df_test.replace(to_replace=' ?',value=np.nan, inplace=True)
df_test.dropna(inplace=True)
df_test.reset_index()

encoders = dict()

for col in categorical_headers:
    df[col] = df[col].str.strip()
    df_train[col] = df_train[col].str.strip()
    df_test[col] = df_test[col].str.strip()
    

    encoders[col] = LabelEncoder()
    df[col+'_int'] = encoders[col].fit_transform(df[col])
    df_train[col+'_int'] = encoders[col].transform(df_train[col])
    df_test[col+'_int'] = encoders[col].transform(df_test[col])
    
    
    
for col in int_categorical_headers:
    df[col+'_int'] = df[col]
    df_train[col+'_int'] = df_train[col]
    df_test[col+'_int'] = df_test[col]

for col in numeric_headers:
    df_train[col] = df_train[col].astype(np.float)
    df_test[col] = df_test[col].astype(np.float)
    df[col] = df[col].astype(np.float)
    ss = StandardScaler()
    df[col] = ss.fit_transform(df[col].values.reshape(-1, 1))
    df_train[col] = ss.transform(df_train[col].values.reshape(-1, 1))
    df_test[col] = ss.transform(df_test[col].values.reshape(-1, 1))

    
df_train.head()

### 1.4 Cross-product Features

After looking at all the useful features in our dataset, we need to find features that can be combined into cross-product features for the purpose of memorization techniques in wide networks. Unfortunately, most of the medications and illnesses in the dataset require extreme domain knowledge, outside our realm of expertise as computer science undergraduates. Despite our lack of knowledge in the medical domain, we are still left with a few options that we can logically group together with common intuition in order to discover and create new features. These include:

- [race, gender]: Race and gender may not be the deciding factor in our model's predictions. However, these two factors are intuitive and could potentially have correlations to different classification outcomes. Furthermore, race and gender cross-products make conceptual sense because certain race-gender combinations can sometimes have varying correlations to diabetic status. This could potentially be due to genetics and socioeconomic factors.

- [age, diag_1]: Age and the patient's primary diagnosis are two other features that seem like they could be logiically grouped together. Crossing age with diagnosis may give more importance to a patient's age when determining how resilient they are to a certain illness and how that impacts their potential readmittance to the hospital. It is clear that in most circumstances, patients with a more advanced age might not react as well as a young adult, given the same illness.

- [gender, diag_1]: Similar to the reasoning to age and primary diagnosis, it might be useful explore a gender-diagnosis cross-product feature. There may be primary diagnoses that vary by each gender.

Though we have labelled instances of Cross-product features we would like to experiment, we thought it might also be worthwhile to mention a cross-product feature type that we would not want to experiment with. In particular, we do not think it would be wise to create cross-product features of the 24 different medications. Though it may be tempting to want to see how different combinations medications might correlate with one another, we simply do not have the domain expertise to authoritatively determine whether two medications may be mixed with each other. If we haphazardly generate cross-product features of two incompatible drugs, it would deviate from a realistic model.



### 1.5 Evaluation Metrics

In determining the most important evaluation metric, it is important to reiterate the business case with our classification model. As we stated before, our binary classification problem aims to let doctors and patients know whether the patient is at risk for a hospital readmission within 30 days of discharge. Given that information, we strongly believe that it is of the utmost importance to reduce the number of false-negatives. In the context of our classification problem, a false-negative is would entail classifying a patient as a "NO" admittance, when they in truth, are at risk for return. Telling a patient that they have nothing to worry about, only to be hospitalized later, can be damaging to all stakeholders at hand. The patient might be dissapointed because they could have taken more precautionary measures despite not being told to, and the doctor's credibility would be damaged. It would be better to overprepare a discharged patient for the worst-case scenario by giving them a false-positive diagnosis.

Given the contextual analysis above of our business case, we believe that the most appropriate evaluation metric for our model would be a Recall scoring model. We have decided to use Recall as our scoring metric because it is equally important whether or not the model's predictions are true-positive or true-negative, but also necessary to penalize the model for giving false-negatives that could underprepare the patient for planning their future health once they are discharged.


### 1.6 Cross Validation Methods

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

uniq, label = np.unique(df['readmitted'], return_inverse=True)

plt.pie(np.bincount(label), labels=uniq, autopct='%1.1f%%')
plt.axis('equal')
plt.title('Class Distribution:')
plt.show()

By looking at our class distribution visualization above, we can tell that we have a large class imbalance. 82.8% of our entries consist of patients who were not readmitted, while 17.2% of our dataset was readmitted. Because we have two unbalanced classes, it is statistically necessary to use a Stratified Split to ensure that each fold has an equal representation of the under-represented class. Because we only have around ~40,000 instances in our training set, compared to the tens of millions of diabetic patients in the United States, it makes sense to perform our cross-validation using a k-folding method. Due to the size limitation of our dataset when compared to the population numbers, we need to maximize the number of instances in each iteration of k-fold. A k = 10 fold cross-validation will utilize 90% of the data in each iteration. For the reasons mentioned above, we believe a 10-fold stratified cross validation will maximize the generalization performance of our trained model.


## 2. Modeling

For our classification model, we will be experimenting with different implementations of neural networks. With our wide network, we will be implementing the cross-product features that we discussed above. For our deep network, we will be experimenting with different numbers of network layers. We will then integrate both wide and deep network architectures into one network and then experiment with different network architectures.

### 2.1 Wide and Deep Network Implementation

We first start by creating a standard combined wide and deep network implementation. 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Input
from keras.layers import Embedding, Flatten, Merge, concatenate
from keras.models import Model
from sklearn import metrics as mt

In [ ]:
master_numeric_headers = numeric_headers + int_categorical_headers
categorical_headers_ints = [x+'_int' for x in categorical_headers]
df_num =  df[numeric_headers].values
X_train_num =  df_train[master_numeric_headers].values
X_test_num =  df_test[master_numeric_headers].values
y_train = df_train['readmitted'].values.astype(np.int)
y_test = df_test['readmitted'].values.astype(np.int)

In [ ]:
# Define cross columns for later. Declare them now in case a cell needs to 
# be rerun, eliminating single cell dependencies

cross_columns = [['gender','race'],
                 ['age', 'diag_1'],
                ['gender', 'diag_1']]

### Standard Configuration Wide and Deep Network

In [ ]:
# we need to create separate sequential models for each embedding
import tensorflow as tf
with tf.device('/GPU:0'):
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_headers_ints:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)
    deep_branch = Dense(units=50,activation='relu')(deep_branch)
    deep_branch = Dense(units=10,activation='relu')(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model1 = Model(inputs=all_inputs, outputs=final_branch)

    model1.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model1.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
yhat1 = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat1),mt.accuracy_score(y_test,yhat1))

### Deeper "Deep" Network

In [ ]:
# we need to create separate sequential models for each embedding
import tensorflow as tf
with tf.device('/GPU:0'):
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_headers_ints:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)
    deep_branch = Dense(units=50,activation='relu')(deep_branch)
    deep_branch = Dense(units=40,activation='relu')(deep_branch)
    deep_branch = Dense(units=30,activation='relu')(deep_branch)
    deep_branch = Dense(units=20,activation='relu')(deep_branch)
    deep_branch = Dense(units=10,activation='relu')(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model2 = Model(inputs=all_inputs, outputs=final_branch)

    model2.compile(optimizer='adagrad',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model2.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
yhat2 = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat2),mt.accuracy_score(y_test,yhat2))

### Deeper Final Stage Network

In [ ]:
# we need to create separate sequential models for each embedding
import tensorflow as tf
with tf.device('/GPU:0'):
    embed_branches = []
    X_ints_train = []
    X_ints_test = []
    all_inputs = []
    all_branch_outputs = []

    for cols in cross_columns:
        # encode crossed columns as ints for the embedding
        enc = LabelEncoder()

        # create crossed labels
        # needs to be commented better, Eric!
        X_crossed_train = df_train[cols].apply(lambda x: '_'.join(x), axis=1)
        X_crossed_test = df_test[cols].apply(lambda x: '_'.join(x), axis=1)

        enc.fit(np.hstack((X_crossed_train.values,  X_crossed_test.values)))
        X_crossed_train = enc.transform(X_crossed_train)
        X_crossed_test = enc.transform(X_crossed_test)
        X_ints_train.append( X_crossed_train )
        X_ints_test.append( X_crossed_test )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name = '_'.join(cols))
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # merge the branches together
    wide_branch = concatenate(all_branch_outputs)

    # reset this input branch
    all_branch_outputs = []
    # add in the embeddings
    for col in categorical_headers_ints:
        # encode as ints for the embedding
        X_ints_train.append( df_train[col].values )
        X_ints_test.append( df_test[col].values )

        # get the number of categories
        N = max(X_ints_train[-1]+1) # same as the max(df_train[col])

        # create embedding branch from the number of categories
        inputs = Input(shape=(1,),dtype='int32', name=col)
        all_inputs.append(inputs)
        x = Embedding(input_dim=N, output_dim=int(np.sqrt(N)), input_length=1)(inputs)
        x = Flatten()(x)
        all_branch_outputs.append(x)

    # also get a dense branch of the numeric features
    all_inputs.append(Input(shape=(X_train_num.shape[1],),sparse=False,name='numeric_data'))
    x = Dense(units=20, activation='relu')(all_inputs[-1])
    all_branch_outputs.append( x )

    # merge the branches together
    deep_branch = concatenate(all_branch_outputs)
    deep_branch = Dense(units=50,activation='relu')(deep_branch)
    deep_branch = Dense(units=10,activation='relu')(deep_branch)

    final_branch = concatenate([wide_branch, deep_branch])
    final_branch = Dense(units=50,activation='sigmoid')(final_branch)
    final_branch = Dense(units=1,activation='sigmoid')(final_branch)

    model3 = Model(inputs=all_inputs, outputs=final_branch)

    model3.compile(optimizer='Adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model3.fit(X_ints_train+ [X_train_num],
            y_train, epochs=10, batch_size=32, verbose=1)

In [ ]:
yhat3 = np.round(model.predict(X_ints_test + [X_test_num]))
print(mt.confusion_matrix(y_test,yhat3),mt.accuracy_score(y_test,yhat3))

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
SVG(model_to_dot(model3).create(prog='dot', format='svg'))

### 2.2 Generalization Performance

### 2.3 Performance Analysis Compared to Scikit-Learn MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

df_dummies = pd.get_dummies(df, columns=categorical_headers)

df_dummies_train, df_dummies_test = train_test_split(df_dummies, test_size=0.2, stratify=df_dummies['readmitted'])
y_train_sk = df_dummies_train['readmitted']
y_test_sk = df_dummies_test['readmitted']
df_dummies_train.drop(['readmitted'], axis=1, inplace=True)
df_dummies_test.drop(['readmitted'], axis=1, inplace=True)

clf = MLPClassifier()
clf.fit(X=df_dummies_train.values, y=np.asarray(y_train_sk.values, dtype="|S6"))

In [ ]:
yhat_sk = clf.predict(df_dummies_test.values)
print(mt.confusion_matrix(y_test_sk.values.astype(int),yhat_sk.astype(int)),mt.accuracy_score(y_test_sk.values.astype(int),yhat_sk.astype(int)))

## 3. Visualizing Embedding Weights